# [Baseline](https://dacon.io/competitions/official/235745/codeshare/2839?page=1&dtype=recent) / 단지별로 잘 묶고, 각 column normalize & discretization

In [ ]:
!pip install optuna
#!pip install lightgbm
#!pip install scikit-learn
#!pip install pycaret

     |████████████████████████████████| 307kB 4.2MB/s 
     |████████████████████████████████| 174kB 39.3MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 143kB 41.9MB/s 
     |████████████████████████████████| 112kB 40.4MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=cbb60851ee7b4d15997518a15c17d216481626dd5755269e0a71ece082a1fb62
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [ ]:
!pip install catboost

     |████████████████████████████████| 69.2MB 39kB/s 


# ※ Data Loading

In [ ]:
from google.colab import drive
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import optuna
from lightgbm import LGBMRegressor
import catboost as cat
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, cross_val_score, KFold

drive.mount('/content/gdrive')

train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/test.csv")

## 결측치 행 삭제
train_dropped = train.dropna(axis=0)

COLS = list(train_dropped.columns)
print(COLS)

Mounted at /content/gdrive
['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수']


**단지코드가 같으면**, <br>
['지역', '임대건물구분', '단지코드', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '공가수', '단지내주차면수', '등록차량수', '총세대수']는 같고<br>
['자격유형', '임대보증금', '전용면적', '임대료', '공급유형', '전용면적별세대수']는 다름

In [ ]:
code_list = list(train_dropped['단지코드'].value_counts().sort_index().index)
previous = COLS

for code in code_list:

  current = []
  tmp = train_dropped[train_dropped['단지코드'] == code ]
  for i in range(len(train_dropped.columns)):

    if len(train_dropped[train_dropped['단지코드'] == code ].iloc[:,i].value_counts().index) == 1:
      current.append(COLS[i])

  previous = list(set(previous).intersection(current))
      
# print(previous)

## 단지코드 같아도 값이 다른 features

difference = list(set(COLS) - set(previous))
print(difference)

['공급유형', '전용면적', '임대료', '자격유형', '전용면적별세대수', '임대보증금']


# ※ Preprocessing
<br>나중엔 normalize랑 discretization도 할 듯

+ onehot encoding하면 dataframe이 너무 sparse해질 수 있다는 점 유의

## 1. Categorical features all dummy화 & '-'값 제거 ver.

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

X_ = pd.get_dummies(data = train_dropped, columns = ['단지코드', '임대건물구분', '지역', '공급유형', '자격유형'])
X_copy = X_.copy()
X = X_copy.drop(["등록차량수"], axis=1)

y = train_dropped['등록차량수']

p = X[X['임대료']=='-'].index
q = X[X['임대료']=='-'].index

droplist = list(set(set(p) | set(q)))

X = X.drop(droplist, axis = 0)
y = y.drop(droplist, axis = 0)

X['임대보증금'] = pd.to_numeric(X['임대보증금'])
X['임대료'] = pd.to_numeric(X['임대료'])

X = X.reset_index(drop = True)
y = y.reset_index(drop = True)

In [ ]:
X

,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,단지코드_C1000,단지코드_C1013,단지코드_C1014,단지코드_C1015,단지코드_C1017,단지코드_C1022,단지코드_C1024,단지코드_C1026,단지코드_C1027,단지코드_C1032,단지코드_C1036,단지코드_C1039,단지코드_C1045,단지코드_C1046,단지코드_C1048,단지코드_C1049,단지코드_C1056,단지코드_C1057,단지코드_C1059,단지코드_C1065,단지코드_C1077,단지코드_C1084,단지코드_C1085,단지코드_C1089,단지코드_C1095,단지코드_C1102,단지코드_C1103,단지코드_C1109,단지코드_C1122,단지코드_C1129,단지코드_C1131,...,단지코드_C2692,임대건물구분_아파트,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_부산광역시,지역_서울특별시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도,공급유형_공공임대(10년),공급유형_공공임대(50년),공급유형_공공임대(5년),공급유형_공공임대(분납),공급유형_국민임대,공급유형_영구임대,공급유형_장기전세,공급유형_행복주택,자격유형_A,자격유형_B,자격유형_C,자격유형_E,자격유형_F,자격유형_G,자격유형_H,자격유형_I,자격유형_J,자격유형_K,자격유형_L,자격유형_M,자격유형_N,자격유형_O
0,900,39.72,134,38.0,15667000,103680,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,900,39.72,15,38.0,15667000,103680,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,900,51.93,385,38.0,27304000,184330,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,900,51.93,15,38.0,27304000,184330,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,900,51.93,41,38.0,27304000,184330,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,239,49.20,19,7.0,11346000,116090,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2281,239,51.08,34,7.0,14005000,142310,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2282,239,51.73,34,7.0,14005000,142310,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2283,239,51.96,114,7.0,14005000,142310,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2. Categorical features all dummy화 (단지코드 삭제) & '-'값 제거 ver.

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

X_ = train_dropped.drop(["등록차량수", "단지코드"], axis=1)
X_2 = pd.get_dummies(data = X_, columns = ['임대건물구분', '지역', '공급유형', '자격유형'])
X = X_2.copy()

y = train_dropped['등록차량수']

p = X[X['임대료']=='-'].index
q = X[X['임대료']=='-'].index

droplist = list(set(set(p) | set(q)))

X = X.drop(droplist, axis = 0)
y = y.drop(droplist, axis = 0)

X['임대보증금'] = pd.to_numeric(X['임대보증금'])
X['임대료'] = pd.to_numeric(X['임대료'])

X = X.reset_index(drop = True)
y = y.reset_index(drop = True)

## 3. Categorical features all dummy화 (단지코드 삭제) & '-'값 제거 & normalization ver.

문제점 : test set은 normalize 불가일 듯
https://soo-jjeong.tistory.com/122

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

X_ = train_dropped.drop(["등록차량수", "단지코드"], axis=1)
X_2 = pd.get_dummies(data = X_, columns = ['임대건물구분', '지역', '공급유형', '자격유형'])
X = X_2.copy()

y = train_dropped['등록차량수']

p = X[X['임대료']=='-'].index
q = X[X['임대료']=='-'].index

droplist = list(set(set(p) | set(q)))

X = X.drop(droplist, axis = 0)
y = y.drop(droplist, axis = 0)

X['임대보증금'] = pd.to_numeric(X['임대보증금'])
X['임대료'] = pd.to_numeric(X['임대료'])

X = X.reset_index(drop = True)
y = y.reset_index(drop = True)

scaler = MinMaxScaler()
X[:] = scaler.fit_transform(X[:])
y[:] = scaler.fit_transform(y[:])

ValueError: ignored

## 4. Categorical features all dummy화 (단지코드 삭제) & '-'값 제거 & standardization ver.

# ※ Modeling

## LGBM & CatBoost

## Optuna (LGBM)

### Revised version (06.26)

Test set에서 33%는 public / 67%는 private에 쓰이는데, 이 때 단지별로는 묶인다는 점 조심!

In [ ]:
unique_test = test.drop_duplicates(subset = ["단지코드"])
unique_test 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
8,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,H,22830000,189840,0.0,3.0,1216.0
17,C1456,619,아파트,부산광역시,국민임대,33.40,82,18.0,A,19706000,156200,0.0,16.0,547.0
26,C1840,593,아파트,전라북도,국민임대,39.57,253,7.0,A,14418000,108130,0.0,3.0,543.0
30,C1332,1297,아파트,경기도,국민임대,39.99,282,11.0,H,28598000,203050,0.0,2.0,1112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,C2456,349,아파트,제주특별자치도,국민임대,26.44,24,17.0,H,6992000,117000,0.0,4.0,270.0
1000,C1266,596,아파트,충청북도,국민임대,26.94,164,35.0,H,8084000,149910,0.0,1.0,593.0
1005,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,-,-,0.0,1.0,40.0
1007,C1267,675,아파트,경상남도,국민임대,24.87,28,38.0,H,6882000,104370,0.0,1.0,467.0


In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

def objective(trial):
  MAE_cv = []
  # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

  params_lgb = {
        "criterion" : trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05),
        "n_estimators": trial.suggest_int("n_estimators", 300, 3000),
        "objective": "regression",
        "metric": "MAE",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500)
  }
  

  kf = KFold(n_splits = 5)
  kf.get_n_splits(X)

  for train_index, valid_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    model = LGBMRegressor(**params_lgb)
    model.fit(
      X_train,
      y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)], # early stopping에 쓰일
      early_stopping_rounds=100,
      verbose= False,
  )

    y_pred = model.predict(X_valid)
    MAE_cv.append( MAE(y_valid, y_pred) )
    #MAE = cross_val_score(model, X_train, y_valid, scoring="neg_mean_absolute_error", n_jobs = -1, cv = 5).mean()
    
  MAE_cv = np.mean(MAE_cv)
  return MAE_cv



sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=50)
print("Best score :", study.best_value)
print("Best params combination :", study.best_trial.params)

[I 2021-06-26 00:14:58,880] A new study created in memory with name: lgbm_parameter_opt
[I 2021-06-26 00:14:59,959] Trial 0 finished with value: 151.36358578728294 and parameters: {'criterion': 'entropy', 'learning_rate': 0.03927975767245621, 'n_estimators': 1916, 'reg_alpha': 4.688999026868671e-06, 'reg_lambda': 0.014039515270313035, 'max_depth': 2, 'num_leaves': 222, 'colsample_bytree': 0.7606690070459252, 'subsample': 0.7956508044572318, 'subsample_freq': 1, 'min_child_samples': 98, 'max_bin': 450}. Best is trial 0 with value: 151.36358578728294.
[I 2021-06-26 00:15:02,023] Trial 1 finished with value: 157.54171149761692 and parameters: {'criterion': 'gini', 'learning_rate': 0.017336180394137354, 'n_estimators': 1121, 'reg_alpha': 1.5747445384650815e-05, 'reg_lambda': 0.03887505735834024, 'max_depth': 6, 'num_leaves': 158, 'colsample_bytree': 0.4836963163912251, 'subsample': 0.5045012539746527, 'subsample_freq': 4, 'min_child_samples': 48, 'max_bin': 436}. Best is trial 0 with value

Best score : 146.51467750942683
Best params combination : {'criterion': 'entropy', 'learning_rate': 0.021083872511859402, 'n_estimators': 1110, 'reg_alpha': 1.695424848820291e-05, 'reg_lambda': 0.017278228677842115, 'max_depth': 3, 'num_leaves': 131, 'colsample_bytree': 0.5397308545447381, 'subsample': 0.5554177854138612, 'subsample_freq': 4, 'min_child_samples': 46, 'max_bin': 371}


## Optuna (CatBoost)

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

def objective(trial):
  MAE_cv = []
  # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

  params_cat = {
        # 'objective': trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        # "n_estimators":trial.suggest_int("n_estimators", 500, 3000),
        'iterations' : trial.suggest_int('iterations', 30, 300),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        'loss_function' : 'MAE',
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),                         
        'depth' : trial.suggest_int('depth', 2, 12),                                       
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.05),              
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        # 'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        #'learning_rate' :trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
  }
  

  kf = KFold(n_splits = 5)
  kf.get_n_splits(X)

  for train_index, valid_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    model = cat.CatBoostRegressor(**params_cat)
    model.fit(
      X_train,
      y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)], # early stopping에 쓰일
      early_stopping_rounds=100,
      verbose= False,
  )

    y_pred = model.predict(X_valid)
    MAE_cv.append( MAE(y_valid, y_pred) )
    #MAE = cross_val_score(model, X_train, y_valid, scoring="neg_mean_absolute_error", n_jobs = -1, cv = 5).mean()
    
  MAE_cv = np.mean(MAE_cv)
  return MAE_cv



sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="cat_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=50)
print("Best score :", study.best_value)
print("Best params combination :", study.best_trial.params)

[I 2021-06-26 00:21:46,744] A new study created in memory with name: cat_parameter_opt
[I 2021-06-26 00:22:40,216] Trial 0 finished with value: 238.49896015880387 and parameters: {'colsample_bylevel': 0.04370861069626263, 'iterations': 287, 'random_strength': 73, 'l2_leaf_reg': 1.7963767941069128e-05, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'depth': 9, 'learning_rate': 0.010823379771832098, 'min_child_samples': 98, 'max_bin': 450, 'od_type': 'IncToDec'}. Best is trial 0 with value: 238.49896015880387.
[I 2021-06-26 00:22:42,298] Trial 1 finished with value: 244.73482343394053 and parameters: {'colsample_bylevel': 0.026506405886809047, 'iterations': 112, 'random_strength': 53, 'l2_leaf_reg': 1.2964031109077052e-05, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'depth': 7, 'learning_rate': 0.041407038455720545, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}. Best is trial 0 with value: 238.49896015880387.
[I 2021-06-26 00:22:45,392] Trial 2 finish

## PyCaret

https://dacon.io/codeshare/1701

In [ ]:
!pip install pycaret

     |████████████████████████████████| 266kB 6.7MB/s 
     |████████████████████████████████| 112kB 10.1MB/s 
     |████████████████████████████████| 1.4MB 8.9MB/s 
     |████████████████████████████████| 276kB 22.7MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 256kB 21.9MB/s 
     |████████████████████████████████| 1.7MB 24.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0MB 39.0MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 6.8MB 32.3MB/s 
     |████████████████████████████████| 14.2MB 157kB/s 
     |████████████████████████████████| 174kB 39.0MB/s 
     |████████████████████████████████| 1.1MB 29.2MB/s 
     |████████████████████████████████| 3.1MB 28.1MB/s 
     |████████████████████████████████| 112kB 39.2MB/

In [ ]:
from pycaret.regression import *

In [ ]:
train_caret = pd.concat([X, y], axis = 1)

In [ ]:
train_caret

,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,단지코드_C1000,단지코드_C1013,단지코드_C1014,단지코드_C1015,단지코드_C1017,단지코드_C1022,단지코드_C1024,단지코드_C1026,단지코드_C1027,단지코드_C1032,단지코드_C1036,단지코드_C1039,단지코드_C1045,단지코드_C1046,단지코드_C1048,단지코드_C1049,단지코드_C1056,단지코드_C1057,단지코드_C1059,단지코드_C1065,단지코드_C1077,단지코드_C1084,단지코드_C1085,단지코드_C1089,단지코드_C1095,단지코드_C1102,단지코드_C1103,단지코드_C1109,단지코드_C1122,단지코드_C1129,단지코드_C1131,...,임대건물구분_아파트,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_부산광역시,지역_서울특별시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도,공급유형_공공임대(10년),공급유형_공공임대(50년),공급유형_공공임대(5년),공급유형_공공임대(분납),공급유형_국민임대,공급유형_영구임대,공급유형_장기전세,공급유형_행복주택,자격유형_A,자격유형_B,자격유형_C,자격유형_E,자격유형_F,자격유형_G,자격유형_H,자격유형_I,자격유형_J,자격유형_K,자격유형_L,자격유형_M,자격유형_N,자격유형_O,등록차량수
0,900,39.72,134,38.0,15667000,103680,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1015.0
1,900,39.72,15,38.0,15667000,103680,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1015.0
2,900,51.93,385,38.0,27304000,184330,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1015.0
3,900,51.93,15,38.0,27304000,184330,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1015.0
4,900,51.93,41,38.0,27304000,184330,0.0,3.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,239,49.20,19,7.0,11346000,116090,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,146.0
2281,239,51.08,34,7.0,14005000,142310,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,146.0
2282,239,51.73,34,7.0,14005000,142310,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,146.0
2283,239,51.96,114,7.0,14005000,142310,0.0,1.0,166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,146.0


In [ ]:
clf = setup(data = train_caret, target = '등록차량수')

,Description,Value
0,session_id,5584
1,Target,등록차량수
2,Original Data,"(2285, 452)"
3,Missing Values,False
4,Numeric Features,451
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1599, 449)"


# ※ Evaluation